In [94]:
import pandas as pd

df_grocery_products = pd.read_csv('Data/grocery_products_clean.csv')
df_shops = pd.read_csv('Data/shops.csv')
df_transactions = pd.read_csv('Data/transactions.csv')
df_user_loyalty = pd.read_csv('Data/user_loyalty.csv')

In [95]:
df_grocery_products.head()

,DEPARTMENT,CATEGORY,SUBCATEGORY,BREADCRUMBS,SKU,PRODUCT_URL,PRODUCT_NAME,BRAND,PRICE_RETAIL
0,Deli,"Bacon, Hot Dogs, Sausage",NaN,"Deli/Bacon, Hot Dogs, Sausage",40495518,https://www.walmart.com/ip/Great-Value-Origina...,"Original Pepperoni Slices, 6 oz",Great Value,2.58
1,Deli,"Bacon, Hot Dogs, Sausage",NaN,"Deli/Bacon, Hot Dogs, Sausage",840884039,https://www.walmart.com/ip/Marketside-Jumbo-Un...,"Jumbo Uncured Angus Beef Franks, 16 oz, 4 Count",Marketside,4.98
2,Deli,"Bacon, Hot Dogs, Sausage",NaN,"Deli/Bacon, Hot Dogs, Sausage",114435065,https://www.walmart.com/ip/Great-Value-Vermont...,"Vermont Maple Premium Sausage, 12 oz, 8 Count",Great Value,2.76
3,Deli,"Bacon, Hot Dogs, Sausage",NaN,"Deli/Bacon, Hot Dogs, Sausage",44391087,https://www.walmart.com/ip/Great-Value-Thick-S...,"Thick Sliced Bacon Hickory Smoked, 16 oz",Great Value,4.48
4,Deli,"Bacon, Hot Dogs, Sausage",NaN,"Deli/Bacon, Hot Dogs, Sausage",36479865,https://www.walmart.com/ip/Sam-s-Choice-Fully-...,"Fully Cooked Uncured Turkey Bacon, 3 oz, 14 s...",Sam's Choice,3.92


In [96]:
# select franchise (use only 1)
# find central position of user based on purchases, by clustering locations (3 clusters) and taking one of the centroids randomly
# find 3 closest shops and randomly choose 1

def select_shop(df_shops, df_transactions, df_user_loyalty, user_id, franchise):
    user_cards = df_user_loyalty[(df_user_loyalty['user']==user_id) & (df_user_loyalty['franchise']==franchise)]['loyalty_card'].values
    user_transactions = df_transactions[df_transactions['loyalty_card'].isin(user_cards)]
    shops = user_transactions['shop'].value_counts()
    shop=shops.keys()[0]
    return shop
    


In [97]:
# find association rules between all users given 1 product_id
import ast
def top_associations(df_transactions, df_user_loyalty, franchise, product_id, n=10):
    # we should filter by year, but maybe later
    cards_franchise = df_user_loyalty[df_user_loyalty['franchise']==franchise]['loyalty_card'].values
    df_transactions_filter_franchise = df_transactions[df_transactions['loyalty_card'].isin(cards_franchise)]
    #transactions_merged_users = pd.merge(df_transactions, df_user_loyalty, how='inner', on='loyalty_card')
    #total_users = transactions_merged_users['user'].nunique()
    number_transactions = df_transactions_filter_franchise.shape[0]
    count_products = {}
    appeared_together = {}
    for i, row in df_transactions_filter_franchise.iterrows():
        for purchase in ast.literal_eval(row['products']):
            
            if purchase[0] not in count_products.keys() and purchase[0] not in appeared_together.keys():
                count_products[purchase[0]] = 0
                appeared_together[purchase[0]] = 0
            count_products[purchase[0]] += 1
            if product_id in [p[0] for p in ast.literal_eval(row['products'])]:
                appeared_together[purchase[0]] += 1
    for product in count_products.keys():
        count_products[product]=count_products[product]/number_transactions
        appeared_together[product]=appeared_together[product]/number_transactions

    support_products = count_products
    list_products = list(support_products.keys())
    list_products.remove(product_id)
    results = pd.DataFrame(index= list_products,columns=['Lift'])
    support_product_id = support_products[product_id]
    for other_product in list_products:
        support_other = support_products[other_product]
        
        support_both = appeared_together[other_product]
        results.at[other_product, 'Lift'] = support_both/(support_product_id*support_other) if support_product_id > 0 and support_other > 0 else 0
    return results.sort_values(by='Lift', ascending=False).head(n)
    

    
    


In [99]:
top_associations(df_transactions, df_user_loyalty, 'Mercadona', 40495518, n=35)

,Lift
278227399,9.247579
52433346,7.656888
42004056,6.513672
890488670,6.253125
10315178,6.253125
484426015,6.051411
510611107,5.627812
619526186,5.599813
814999346,5.583147
10450992,5.517463


In [100]:
def get_top_products_user(df_transactions, df_user_loyalty, user_id, franchise, n=10):
    # we should do tfidf, but maybe later
    cards_franchise = df_user_loyalty[(df_user_loyalty['user']==user_id) & (df_user_loyalty['franchise']==franchise)]['loyalty_card'].values
    df_transactions_filter_franchise = df_transactions[df_transactions['loyalty_card'].isin(cards_franchise)]
    number_transactions = df_transactions_filter_franchise.shape[0]
    #print(number_transactions)
    count_products = {}
    for i, row in df_transactions_filter_franchise.iterrows():
        for purchase in ast.literal_eval(row['products']):
            if purchase[0] not in count_products.keys():
                count_products[purchase[0]] = 0
            count_products[purchase[0]] += 1
    for product in count_products.keys():
        count_products[product]=count_products[product]/number_transactions
    return pd.DataFrame.from_dict(count_products, orient='index', columns=['Support']).sort_values(by='Support', ascending=False).head(n)

In [101]:
get_top_products_user(df_transactions, df_user_loyalty, 'User2', 'Carrefour', n=30)

,Support
330307128,0.039711
478679077,0.036101
129427454,0.036101
665213079,0.032491
510611107,0.032491
770785583,0.028881
36319397,0.028881
660340986,0.028881
10450119,0.028881
10448531,0.028881


In [102]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_similar_products(df_grocery_products, product_id, n=10):
    # Ensure the product_id exists in the dataframe
    if product_id not in df_grocery_products['SKU'].values:
        return None  # Return an empty DataFrame if product_id is not found
    
    # Find the product name for the given product_id
    product_name = df_grocery_products.loc[df_grocery_products['SKU'] == product_id, 'PRODUCT_NAME'].values[0]
    
    # Generate embeddings for the target product name
    product_embedding = model.encode([product_name])
    
    # Generate embeddings for all product names in the dataframe
    product_names = df_grocery_products['PRODUCT_NAME'].tolist()
    embeddings = model.encode(product_names)
    
    # Calculate cosine similarity between the target product and all products
    similarities = cosine_similarity(product_embedding, embeddings).flatten()
    
    # Add similarity scores to the dataframe
    df_grocery_products['similarity'] = similarities
    
    # Sort by similarity and return the top n similar products
    return df_grocery_products.sort_values(by='similarity', ascending=False).head(n + 1).iloc[1:]

In [103]:
print(get_similar_products(df_grocery_products, 40495518, n=10))

     DEPARTMENT                       CATEGORY SUBCATEGORY  \
7          Deli       Bacon, Hot Dogs, Sausage         NaN   
245      Pantry                     Condiments    Shop all   
547        Deli                    Fresh Pizza         NaN   
136      Pantry              Canned vegetables    Shop all   
555        Deli                    Fresh Pizza         NaN   
657      Frozen  Frozen Pizza, Pasta, & Breads    Shop all   
660      Frozen  Frozen Pizza, Pasta, & Breads    Shop all   
548        Deli                    Fresh Pizza         NaN   
1163       Deli         Prepared Meals & Sides         NaN   
550        Deli                    Fresh Pizza         NaN   

                               BREADCRUMBS        SKU  \
7            Deli/Bacon, Hot Dogs, Sausage  483646515   
245                      Pantry/Condiments  353157052   
547                       Deli/Fresh Pizza  579824441   
136               Pantry/Canned vegetables  863905651   
555                       Deli/F

In [107]:
select_shop(df_shops, df_transactions, df_user_loyalty, 'User2', 'Carrefour')

108

In [115]:
import random
from tqdm import tqdm

def get_product_recommendations(df_transactions, df_user_loyalty, df_grocery_products, user_id, franchise, n=10):

    # recommendations for cross-selling

    # Step 1: Get the top products for the user
    top_products = get_top_products_user(df_transactions, df_user_loyalty, user_id, franchise, n=30)
    #print(top_products)
    # Step 2: Find the top associations for 10 of those products
    recommendations_cross = []
    for product_id in tqdm(top_products.sample(10).index):
        recommendations_cross.extend(top_associations(df_transactions, df_user_loyalty, franchise, product_id, n=10).index)
    
    # filter products already bought by user
    recommendations_cross = list(set(recommendations_cross))
    recommendations_cross = [product for product in recommendations_cross if product not in top_products.index]

    recommendations_cross = random.sample(recommendations_cross, n) if len(recommendations_cross) > n else recommendations_cross 

    # recommendation for upselling

    recommendations_upsell = []
    for product_id in tqdm(top_products.sample(10).index):
        #print(product_id)
        similar_products = get_similar_products(df_grocery_products, product_id, n=3)['SKU'].tolist()
        # filter products with heigher price
        #print('Similar:', similar_products)
        similar_products = [product for product in similar_products if df_grocery_products.loc[df_grocery_products['SKU'] == product, 'PRICE_RETAIL'].values[0] > df_grocery_products.loc[df_grocery_products['SKU'] == product_id, 'PRICE_RETAIL'].values[0]]
        recommendations_upsell.extend(similar_products)
    
    # filter products already bought by user
    recommendations_upsell = list(set(recommendations_upsell))
    recommendations_upsell = [product for product in recommendations_upsell if product not in top_products.index]

    recommendations_upsell = random.sample(recommendations_upsell, n) if len(recommendations_upsell) > n else recommendations_upsell

    # recommendation for more frequent visits
    recommendation_frequent = top_products[5:15].sample(2).index.tolist()

    
    
    return recommendations_cross, recommendations_upsell, recommendation_frequent

In [116]:
get_product_recommendations(df_transactions, df_user_loyalty, df_grocery_products, 'User2', 'Carrefour', n=10)

100%|██████████| 10/10 [00:23<00:00,  2.33s/it]


([20574656,
  48714473,
  102905634,
  568548658,
  932430893,
  807765818,
  39127387,
  539369459,
  133141258,
  10534285],
 [481640762,
  953343781,
  406512269,
  51762226,
  913337173,
  31091075,
  29933424,
  515786216,
  10295202,
  412509991],
 [770785583, 10314925])

KeyError: 'image'